In [1]:
%load_ext lab_black

In [2]:
from pyprojroot import here
import os
from skimpy import clean_columns
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
import shap
import joblib
from joblib import load

path_data = here("./data")
os.chdir(path_data)

data = pd.read_csv("data_er_visits.csv").rename(columns={"Unnamed: 0": "Member ID"})
data_no_member_id = data.drop(columns=["Member ID"])
data_predict = data_no_member_id.drop(columns=["Hospital ID", "ER Visit"])

# Load the model
model = joblib.load("model_drivers.joblib")

SystemError: initialization of _internal failed without raising an exception

Get the actual percentage of members with ER visits in the last 30 days for each hospital

Show them their average percentages for all four variables.

Add some noise for the features by Hospital ID

Then create a function to allow for simulated data with the exact percentage that someone shows

In [4]:
data_no_member_id.groupby("Hospital ID").mean().reset_index().round(2)

,Hospital ID,Age 60+,High Cholesterol,Diabetes,Preventative Services,ER Visit
0,1,0.29,0.41,0.25,0.51,0.28
1,2,0.28,0.40,0.24,0.50,0.27
2,3,0.29,0.40,0.24,0.49,0.28


Step: Create a function that generates 

In [7]:
import pandas as pd
import numpy as np


def adjust_binary_percentages(df, **column_percentages):
    """
    Adjusts the percentage of 1's in each specified column of a binary DataFrame.

    Args:
    df (pd.DataFrame): DataFrame with binary values.
    column_percentages (dict): A dictionary where keys are column names and values are the new desired percentages of 1's.

    Returns:
    pd.DataFrame: Modified DataFrame.
    """

    for column, percentage in column_percentages.items():
        if column not in df.columns:
            raise ValueError(f"Column {column} not found in DataFrame")

        # Calculate current percentage of 1's
        current_percentage = df[column].mean()

        # Calculate the desired number of 1's
        target_count = int(df.shape[0] * percentage)

        # Find indices where changes are needed
        ones_indices = df[df[column] == 1].index
        zeros_indices = df[df[column] == 0].index

        if target_count > ones_indices.size:  # Need to add more 1's
            change_count = target_count - ones_indices.size
            indices_to_change = np.random.choice(
                zeros_indices, change_count, replace=False
            )
            df.loc[indices_to_change, column] = 1
        else:  # Need to remove some 1's
            change_count = ones_indices.size - target_count
            indices_to_change = np.random.choice(
                ones_indices, change_count, replace=False
            )
            df.loc[indices_to_change, column] = 0

    return df

In [10]:
data_predict_adjust = clean_columns(data_predict.copy())
adjust_binary_percentages(
    df=data_predict_adjust, age_60 = .4, high_cholesterol = .5, 
)

,age_60,high_cholesterol,diabetes,preventative_services
0,0,0,0,1
1,1,0,0,0
2,1,0,0,1
3,0,1,0,0
4,0,0,0,1
...,...,...,...,...
9995,1,1,1,1
9996,1,0,0,1
9997,1,0,0,0
9998,0,0,0,0


In [11]:
import xgboost
import numpy as np
import pandas as pd
from sklearn.inspection import plot_partial_dependence
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Creating a synthetic dataset
X, y = np.random.rand(100, 5), np.random.randint(0, 2, 100)
X = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(1, 6)])
y = pd.Series(y, name='target')

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training an XGBoost model
model = xgboost.XGBClassifier()
model.fit(X_train, y_train)

# Selecting the feature for which we want to plot the ICE plot
feature_to_plot = 'feature_1'

# Generating the ICE plot
fig, ax = plt.subplots(figsize=(10, 6))
plot_partial_dependence(model, features=[feature_to_plot], X=X_train, ax=ax, kind='individual', grid_resolution=20)
plt.show()


[14:38:43] WARNING: /mnt/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.


TypeError: plot_partial_dependence() got an unexpected keyword argument 'kind'

In [5]:
# Make predictions (opxtional, to evaluate model)
predictions = model.predict(data_predict)

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
